**Chapter 5: Building a Chatbot Like ChatGPT**

In [2]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

from config import set_environment
set_environment()

## Understanding retrieval and vectors

### Embeddings

In [2]:
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
text = "This is a sample query."
query_result = embeddings.embed_query(text)
print(query_result[:10])
print(len(query_result))

[-0.009221134094452037, 0.006869847429662192, -0.006328029137472195, -0.008716800602076323, -0.027084101053996855, 0.02807913720245218, -0.012962747434994147, -0.004613974756300044, -0.02716588420912845, -0.027111362726589063]
1536


In [3]:
from langchain.embeddings.openai import OpenAIEmbeddings
words = ["cat", "dog", "computer", "animal"]
embeddings = OpenAIEmbeddings()
doc_vectors = embeddings.embed_documents(words)

In [4]:
from scipy.spatial.distance import pdist, squareform
import numpy as np
import pandas as pd
X = np.array(doc_vectors)
dists = squareform(pdist(X))

In [5]:
import pandas as pd

df = pd.DataFrame(
    data=dists,
    index=words,
    columns=words
)
df.style.background_gradient(cmap='coolwarm')

,cat,dog,computer,animal
cat,0.000000,0.523532,0.576484,0.522346
dog,0.523532,0.000000,0.583924,0.481125
computer,0.576484,0.583924,0.000000,0.594266
animal,0.522346,0.481125,0.594266,0.000000


### Vector Storage

In [6]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

In [7]:
from langchain.document_loaders import ArxivLoader
from langchain.text_splitter import CharacterTextSplitter

loader = ArxivLoader(query="2310.06825")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [8]:
# vectorstore = Chroma.from_documents(documents=docs, embedding=OpenAIEmbeddings())
# similar_vectors = vector_store.query(query_vector, k)

## Loading and retrieving in LangChain

### Document loaders

In [9]:
from langchain.document_loaders import WikipediaLoader
loader = WikipediaLoader("LangChain")
documents = loader.load()

### Retrievers in LangChain

* kNN retriever

In [10]:
from langchain.retrievers import KNNRetriever
from langchain.embeddings import OpenAIEmbeddings

words = ["cat", "dog", "computer", "animal"]
retriever = KNNRetriever.from_texts(words, OpenAIEmbeddings())

In [11]:
result = retriever.get_relevant_documents("dog")
print(result)

[Document(page_content='dog', metadata={}), Document(page_content='animal', metadata={}), Document(page_content='cat', metadata={}), Document(page_content='computer', metadata={})]


* PubMed retriever

In [12]:
from langchain.retrievers import PubMedRetriever
retriever = PubMedRetriever()
documents = retriever.get_relevant_documents("COVID")
len(documents)

3

In [13]:
for document in documents:
    print(document.metadata["Title"])

Effects on Quality of Life of a Telemonitoring Platform amongst Patients with Cancer (EQUALITE): A Randomized Trial Protocol.
A Virtual Case Presentation Platform: Protocol Study.
A Vulnerability Index to Assess the Risk of SARS-CoV-2-Related Hospitalization/Death: Urgent Need for an Update after Diffusion of Anti-COVID Vaccines.


## Implementing a chatbot

### Document loader

In [14]:
from typing import Any
from langchain.document_loaders import (
    PyPDFLoader, TextLoader,
    UnstructuredWordDocumentLoader,
    UnstructuredEPubLoader
)

In [15]:
class EpubReader(UnstructuredEPubLoader):
    def __init__(self, file_path: str | list[str], **kwargs: Any):
        super().__init__(file_path, **kwargs, mode="elements", strategy="fast")
        
class DocumentLoaderException(Exception):
    pass
    
class DocumentLoader(object):
    """Loads in a document with a supported extension."""
    supported_extentions = {
        ".pdf": PyPDFLoader,
        ".txt": TextLoader,
        ".epub": EpubReader,
        ".docx": UnstructuredWordDocumentLoader,
        ".doc": UnstructuredWordDocumentLoader
    }

In [16]:
import logging
import pathlib
from langchain.schema import Document

def load_document(temp_filepath: str) -> list[Document]:
    """Load a file and return it as a list of documents."""
    ext = pathlib.Path(temp_filepath).suffix
    loader = DocumentLoader.supported_extentions.get(ext)
    if not loader:
        raise DocumentLoaderException(
            f"Invalid extension type {ext}, cannot load this type of file"
        )
        
    loader = loader(temp_filepath)
    docs = loader.load()
    logging.info(docs)
    return docs

### Vector storage

In [17]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.schema import Document, BaseRetriever
from langchain.retrievers.document_compressors import EmbeddingsFilter
from langchain.retrievers import ContextualCompressionRetriever

def configure_retriever(
        docs: list[Document],
        use_compression: bool = False
    ) -> BaseRetriever:
    """"Retriever to use."""
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)
    splits = text_splitter.split_documents(docs)
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    vectordb = DocArrayInMemorySearch.from_documents(splits, embeddings)
    retriever = vectordb.as_retriever(search_type="mmr", search_kwargs={"k": 2, "fetch_k": 4})
    if not use_compression:
        return retriever
    
    embeddings_filter = EmbeddingsFilter(
        embeddings=embeddings, similarity_threshold=0.76
    )
    return ContextualCompressionRetriever(
        base_compressor=embeddings_filter,
        base_retriever=retriever
    )

In [18]:
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.base import Chain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory

def configure_chain(retriever: BaseRetriever) -> Chain:
    """Configure chain with a retriever."""
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    llm = ChatOpenAI(
        model_name="gpt-3.5-turbo", temperature=0, streaming=True
    )
    return ConversationalRetrievalChain.from_llm(
        llm, retriever=retriever, memory=memory, verbose=True, max_tokens_limit=4000
    )

In [19]:
import os
import tempfile

def configure_qa_chain(uploaded_files):
    """Read documents, configure retriever, and the chain."""
    docs = []
    temp_dir=tempfile.TemporaryDirectory()
    for file in uploaded_files:
        temp_filepath = os.path.join(temp_dir.name, file.name)
        with open(temp_filepath, "wb") as f:
            f.write(file.getvalue())
        docs.extend(load_document(temp_filepath))
        
    retriever = configure_retriever(docs=docs)
    return configure_chain(retriever=retriever)

In [ ]:
import streamlit as st
from langchain.callbacks import StreamlitCallbackHandler

st.set_page_config(page_title="LangChain: Chat with Documents", page_icon="🦜")
st.title("🦜 LangChain: Chat with Documents")

uploaded_files = st.sidebar.file_uploader(
    label="Upload files",
    type=list(DocumentLoader.supported_extentions.keys()),
    accept_multiple_files=True
)
if not uploaded_files:
    st.info("Please upload documents to continue.")
    st.stop()
    
qa_chain = configure_qa_chain(uploaded_files)
assistant = st.chat_message("assistant")
user_query = st.chat_input(placeholder="Ask me anything!")

if user_query:
    stream_handler = StreamlitCallbackHandler(assistant)
    response = qa_chain.run(user_query, callbacks=[stream_handler])
    st.markdown(response)

### Memory

**Conversation buffers**

In [3]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI

memory = ConversationBufferMemory()
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo", temperature=0, streaming=True
)

chain = ConversationChain(llm=llm, memory=memory)

user_input = "Hi, how are you?"
response = chain.predict(input=user_input)
print(response)

user_input = "What's the weather like today?"
response = chain.predict(input=user_input)
print(response)

print(memory.chat_memory.messages)

Hello! I'm doing well, thank you for asking. I've been busy processing a lot of information and learning new things. How can I assist you today?
The weather today is partly cloudy with a high of 75 degrees Fahrenheit and a slight chance of rain in the afternoon. It's a great day to enjoy some outdoor activities! Is there anything else you would like to know?
[HumanMessage(content='Hi, how are you?', additional_kwargs={}, example=False), AIMessage(content="Hello! I'm doing well, thank you for asking. I've been busy processing a lot of information and learning new things. How can I assist you today?", additional_kwargs={}, example=False), HumanMessage(content="What's the weather like today?", additional_kwargs={}, example=False), AIMessage(content="The weather today is partly cloudy with a high of 75 degrees Fahrenheit and a slight chance of rain in the afternoon. It's a great day to enjoy some outdoor activities! Is there anything else you would like to know?", additional_kwargs={}, exa

**Remembering conversation summaries**

In [6]:
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

memory = ConversationSummaryMemory(llm=ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0))
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.load_memory_variables({})

{'history': 'The human greets the AI with a simple "hi" and the AI responds by asking "what\'s up."'}

**Storing knowledge graphs**

In [8]:
from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0)
memory = ConversationKGMemory(llm=llm)

**Combining several memory mechanisms**

In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory, CombinedMemory, ConversationSummaryMemory

# Initialize language model (with desired temperature parameter)
llm = ChatOpenAI(temperature=0)
# Define Conversation Buffer Memory (for retaining all past messages)
conv_memory = ConversationBufferMemory(memory_key="chat_history_lines", input_key="input")
# Define Conversation Summary Memory (for summarizing conversation)
summary_memory = ConversationSummaryMemory(llm=llm, input_key="input")
# Combine both memory types
memory = CombinedMemory(memories=[conv_memory, summary_memory])
# Define Prompt Template
_DEFAULT_TEMPLATE = """The following is a friendly conversation between a
human and an AI. The AI is talkative and provides lots of specific details
from its context. If the AI does not know the answer to a question, it
truthfully says it does not know.
Summary of conversation:
{history}
Current conversation:
{chat_history_lines}
Human: {input}
AI:"""

PROMPT = PromptTemplate(input_variables=["history", "input", "chat_history_lines"], template=_DEFAULT_TEMPLATE)
# Initialize the Conversation Chain
conversation = ConversationChain(llm=llm, verbose=True, memory=memory, prompt=PROMPT)
# Start the conversation
conversation.run("Hi!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a
human and an AI. The AI is talkative and provides lots of specific details
from its context. If the AI does not know the answer to a question, it
truthfully says it does not know.
Summary of conversation:

Current conversation:

Human: Hi!
AI:

> Finished chain.


'Hello! How are you today?'

In [12]:
conversation.run("Briefly explain to me what is Generative AI?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a
human and an AI. The AI is talkative and provides lots of specific details
from its context. If the AI does not know the answer to a question, it
truthfully says it does not know.
Summary of conversation:
The human greets the AI with a simple "Hi." The AI responds with a friendly "Hello!" and asks how the human is feeling today.
Current conversation:
Human: Hi!
AI: Hello! How are you today?
Human: Briefly explain to me what is Generative AI?
AI:

> Finished chain.


'Generative AI refers to artificial intelligence systems that are capable of creating new content, such as images, text, or music, based on patterns and data they have been trained on. These systems use algorithms to generate original content that mimics human creativity and can be used in various applications, such as art, design, and even storytelling. Generative AI has been used in fields like computer vision, natural language processing, and creative arts to produce realistic and novel outputs.'

In [13]:
conversation.run("How does it differ from traditional AI and Machine Learning?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a
human and an AI. The AI is talkative and provides lots of specific details
from its context. If the AI does not know the answer to a question, it
truthfully says it does not know.
Summary of conversation:
The human greets the AI with a simple "Hi." The AI responds with a friendly "Hello!" and asks how the human is feeling today. The conversation shifts to Generative AI, with the AI explaining that it refers to artificial intelligence systems that can create new content based on patterns and data they have been trained on. These systems use algorithms to generate original content that mimics human creativity and can be used in various applications. Generative AI has been utilized in fields like computer vision, natural language processing, and creative arts to produce realistic and novel outputs.
Current conversation:
Human: Hi!
AI: Hello! How are you today?
Human: Bri

'Traditional AI and Machine Learning typically involve using algorithms to perform specific tasks based on predefined rules and patterns. In contrast, Generative AI goes beyond this by creating new content that is not explicitly programmed. It can generate original and creative outputs by learning from large datasets and identifying patterns to produce novel content. Generative AI is more focused on creativity and innovation, while traditional AI and Machine Learning are more task-oriented and rule-based.'

**Long-term persistence**

## Moderating responses

In [14]:
from langchain.chains import OpenAIModerationChain
moderation_chain = OpenAIModerationChain()

In [15]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema import StrOutputParser
cot_prompt = PromptTemplate.from_template(
    "{question} \nLet's think step by step!"
)
llm_chain = cot_prompt | ChatOpenAI() | StrOutputParser()

In [16]:
chain = llm_chain | moderation_chain

In [17]:
response = chain.invoke({"question": "What is the future of programming?"})

In [18]:
print(response)

{'input': '1. Continued growth in demand: As technology becomes more integrated into everyday life, the demand for skilled programmers will continue to grow. From developing new software and applications to maintaining and updating existing systems, programmers will be essential in all industries.\n\n2. Specialization: As the field of programming becomes more complex, we may see a trend towards specialization. Programmers may focus on specific languages, platforms, or industries in order to develop expertise in a particular area.\n\n3. Automation and AI: With the rise of automation and artificial intelligence, there may be a shift towards programming machines to perform tasks that were previously done by humans. This could lead to the development of new programming languages and techniques tailored for AI and machine learning.\n\n4. Increased collaboration: Programming is becoming more collaborative, with teams of programmers working together on projects. This trend is likely to contin